# Imports and Installs

In [ ]:
# install latest version of polyglot language detection library for stretch
# !pip install -U git+https://github.com/aboSamoor/polyglot.git@master

# !pip install plotly

In [1]:
import pandas as pd
import numpy as np 
#import pandas_profiling 
from sklearn import preprocessing # for category encoder
# from polyglot.detect import Detector # for language detection stretch
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
# much more efficient for larger files like Nearest Neighbors which the model
import joblib
import plotly.express as px
import plotly.graph_objects as go

# EDA & PROFILE REPORT

In [19]:
# df = pd.read_csv('https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv')
# df = df.dropna()
# df = df.drop_duplicates(subset=['track_name', 'artist_name'])

# df.shape

# df.to_csv('spotify_unique_track_id.csv', index=False)
# Data already has duplicate track names and nulls removed.
df = pd.read_csv('https://raw.githubusercontent.com/Build-Week-Spotify-Song-Suggester-5/Data-Science/master/spotify_unique_track_id.csv')
df.shape
# raw_df = raw_df.drop_duplicates(subset='track_name')
# raw_df.shape
# raw_df.to_csv('spotify_unique_track_id.csv', index=False)

(176514, 18)

In [20]:
df.head(1)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.91,0.0,C#,0.346,-1.828,Major,0.0525,166.969,4/4,0.814


# Visualizatoin Stretch Goal Stuff

In [ ]:
# def create_web_chart(df):
#     music_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']
    
#     fig = go.Figure()

#     for idx, row in df.iterrows():
#         # vals = list(df.iloc[idx][music_features].values)
#         row_df = pd.DataFrame(df.loc[idx][music_features].T).reset_index().rename(columns={idx:'values', 'index':'feature'})
#         fig.add_trace(go.Scatterpolar(
#                                     r=row_df['values'],
#                                     theta=row_df['feature'],
#                                     fill='toself',
#                                     name= f'{row.artist_name} - {row.track_name}'
#                     ))
#     # fig.show()
#     return fig
# fig = create_web_chart(df.iloc[0:5])
# fig.show()


# first_song_values = list(df.iloc[0][music_features].values)
# second_song_values = list(df.iloc[1][music_features].values)
# fig = px.bar(first_song_df, x='feature', y="values", color='feature')
# fig.show()



# fig.add_trace(go.Scatterpolar(
#       r=second_song_df['values'],
#       theta=second_song_df['feature'],
#       fill='toself',
#       name='Second Song'
# ))

# fig.update_layout(
#   polar=dict(
#     radialaxis=dict(
#       visible=True,
#       range=[0, 1]
#     )),
#   showlegend=True
# )

# fig.show()

# DATA ENGINEERING CATEGORY ENCODING OF OBJECT FEATURES
note: might need to create another column of encoded languages
resources below: https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/


In [ ]:
# # test of polyglot detector

# arabic_text = u"""
# أفاد مصدر امني في قيادة عمليات صلاح الدين في العراق بأن " القوات الامنية تتوقف لليوم
# الثالث على التوالي عن التقدم الى داخل مدينة تكريت بسبب
# انتشار قناصي التنظيم الذي يطلق على نفسه اسم "الدولة الاسلامية" والعبوات الناسفة
# والمنازل المفخخة والانتحاريين، فضلا عن ان القوات الامنية تنتظر وصول تعزيزات اضافية ".
# """

# detector = Detector(arabic_text)
# print(detector.language)


#    TODO: figure out UTF-8 ERROR or just do try catch exception and return 
#    unknown or something
#    below code does not work


# df.track_name = df.track_name.astype(str)
# df['poly_obj'] = df.track_name.apply(lambda x: Detector(x, quiet=True))
# df['Track-lang'] = df['poly_obj'].apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
# df['Track-LangConfidence'] = df['poly_obj'].apply( lambda x: x.language.confidence)

# ENCODING OF FEATURES

NOTE THAT IF YOU RUN THIS CELL AGAIN WITHOUT RESTARTING RUNTIME YOU'LL ENCODE AND BIND ANOTHER COPY OF GENRE ENCODINGS SIGNIFICANTLY INCREASING THE SIZE OF THE FEATURE SET WHILE ADDING NEGATIVE VALUE

In [21]:
# helper function to one hot encode genre
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

time_sig_encoding = { '0/4' : 0, '1/4' : 1, 
                         '3/4' : 3, '4/4' : 4,
                         '5/4' : 5}

key_encoding = { 'A' : 0, 'A#' : 1, 'B' : 2,
                    'C' : 3,  'C#' : 4,  'D' : 5,
                    'D#' : 6, 'E' : 7, 'F' : 8,
                    'F#' : 9, 'G' : 10, 'G#' : 11 }

mode_encoding = { 'Major':0, 'Minor':1}      

df['key'] = df['key'].map(key_encoding)
df['time_signature'] = df['time_signature'].map(time_sig_encoding)
df['mode'] = df['mode'].map(mode_encoding)
df = encode_and_bind(df, 'genre')    

In [22]:
# df.genre.value_counts()
df.head(2)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,genre_Pop,genre_R&B,genre_Rap,genre_Reggae,genre_Reggaeton,genre_Rock,genre_Ska,genre_Soul,genre_Soundtrack,genre_World
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.0,...,0,0,0,0,0,0,0,0,0,0


# MODELING: Nearest Neighbors
resources: https://scikit-learn.org/stable/modules/neighbors.html

In [39]:
# instantiate model
neigh = NearestNeighbors(n_neighbors=25)

features = list(df.columns[4:])
X = df[features].values
# y = df[target]

X.shape # y.shape
# target = 'track_id'

%time
neigh.fit(X) # NN doesn't need to fit Y

Wall time: 0 ns


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=25, p=2,
                 radius=1.0)

# Manual Model Test on 1 Song Example

In [41]:
# highest ranking popularity song from Ariana Grande
# df.sort_values('popularity', ascending=False)

test_song = X[8980]
df.iloc[8980]

genre                                      Dance
artist_name                        Ariana Grande
track_name                               7 rings
track_id                  14msK75pk3pA33pzPVNtBF
popularity                                   100
acousticness                               0.578
danceability                               0.725
duration_ms                               178640
energy                                     0.321
instrumentalness                               0
key                                            4
liveness                                  0.0884
loudness                                 -10.744
mode                                           1
speechiness                                0.323
tempo                                     70.142
time_signature                                 4
valence                                    0.319
genre_A Capella                                0
genre_Alternative                              0
genre_Anime         

In [25]:
distance, neighbors = neigh.kneighbors(np.array([test_song]))
distance, neighbors

(array([[ 0.        , 14.60647945, 15.83732863, 17.18483261, 18.99379891,
         19.89857749, 20.15658815, 20.16113779, 20.77844259, 21.28437267,
         21.44113734, 22.34923616, 23.02309785, 23.09522555, 24.43566688,
         24.5369614 , 25.25990963, 25.39619324, 25.93901583, 26.68930665,
         26.75912711, 28.22857614, 28.49794995, 29.11360625, 29.22800217,
         29.39026299, 29.84156086, 30.88954642, 32.79003974, 33.33631706,
         33.9356995 , 34.59453678, 35.34497122, 36.08540001, 36.40618131,
         37.98235756, 38.21238371, 38.69293532, 38.76301554, 39.47355762,
         39.69893395, 40.07321411, 40.43581361, 41.14007118, 42.03048018,
         42.08744502, 42.23487786, 42.54548239, 43.04775896, 43.28142343]]),
 array([[  8556,   3619,  38263, 123434,  49315,  58654, 165987,  75302,
         166094,  76351, 101568,  18828, 138345,   6036,  50759, 100003,
         148763,  45754, 116618,   7208,  82508, 175040, 167094, 165133,
          33422, 110258, 105330, 14637

In [26]:
song_list = []
for idx in neighbors[0][2:]: # this way excludes itself
    row = df.iloc[idx]
    print(f'Artist: {row.artist_name} - Track: {row.track_name}')
    song_list.append((row.artist_name, row.track_name))
song_list

Artist: Nonsens - Track: Bounce
Artist: Altafulla - Track: Equivócate Conmigo
Artist: Brother Dege - Track: Hard Row to Hoe
Artist: Imaginary Future - Track: As Long as I Have You I'm Home
Artist: Micky & The Motorcars - Track: One In A Lifetime Girl
Artist: Jungle - Track: Give Over
Artist: Dierks Bentley - Track: Pick Up
Artist: Chris Brown - Track: Girl You Loud
Artist: MiyaGi - Track: Сонная лощина
Artist: Rihanna - Track: Break It Off
Artist: Dick Van Dyke - Track: Fidelity Fiduciary Bank - From "Mary Poppins"/Soundtrack Version
Artist: Palaye Royale - Track: You'll Be Fine
Artist: Oblivians - Track: Call the Police
Artist: The Abyssinians - Track: Declaration of Rights
Artist: Dick Van Dyke - Track: Fidelity Fiduciary Bank
Artist: Blues Saraceno - Track: Devils Got You Beat
Artist: El Alfa - Track: Llego el Moreno
Artist: Jelly Roll - Track: Goodnight Nashville
Artist: Tedashii - Track: Smile
Artist: The Delfonics - Track: Tell Me This Is a Dream
Artist: Jelly Roll - Track: Bad T

[('Nonsens', 'Bounce'),
 ('Altafulla', 'Equivócate Conmigo'),
 ('Brother Dege', 'Hard Row to Hoe'),
 ('Imaginary Future', "As Long as I Have You I'm Home"),
 ('Micky & The Motorcars', 'One In A Lifetime Girl'),
 ('Jungle', 'Give Over'),
 ('Dierks Bentley', 'Pick Up'),
 ('Chris Brown', 'Girl You Loud'),
 ('MiyaGi', 'Сонная лощина'),
 ('Rihanna', 'Break It Off'),
 ('Dick Van Dyke',
  'Fidelity Fiduciary Bank - From "Mary Poppins"/Soundtrack Version'),
 ('Palaye Royale', "You'll Be Fine"),
 ('Oblivians', 'Call the Police'),
 ('The Abyssinians', 'Declaration of Rights'),
 ('Dick Van Dyke', 'Fidelity Fiduciary Bank'),
 ('Blues Saraceno', 'Devils Got You Beat'),
 ('El Alfa', 'Llego el Moreno'),
 ('Jelly Roll', 'Goodnight Nashville'),
 ('Tedashii', 'Smile'),
 ('The Delfonics', 'Tell Me This Is a Dream'),
 ('Jelly Roll', 'Bad Times Roll'),
 ('Tim McGraw', 'Neon Church'),
 ('Utsu-P', '未来の夏休み'),
 ('Oscar Peterson', 'Stormy Weather'),
 ('Wookiefoot', 'All Good'),
 ('Suburban Legends', 'Love Fair'

In [31]:
only_10

[('Buena Vista Social Club', '¿Y Tú Qué Has Hecho? - Live'),
 ('Dierks Bentley', 'Pick Up'),
 ('Tim McGraw', 'Neon Church'),
 ('Dick Van Dyke', 'Fidelity Fiduciary Bank'),
 ('Jonn Hart', 'LIFL'),
 ('El Alfa', 'Llego el Moreno'),
 ('Three Houses Down', "She's My Woman (feat. Spawn Breezie)"),
 ('The Civil Wars', 'From This Valley'),
 ('Lorie', 'Le bonheur a tout prix'),
 ('Oblivians', 'Call the Police')]

# Helper Function to Return 10 Nearest Neighbors

In [42]:
"""
function for what was done manually above
generalized for other dataframes instead of hard coded therefore:
takes in a dataframe, a feature set array, and a song ID
and returns a list of tuples of artist name and track names
"""
from typing import List, Tuple
import random 

def closest_ten(df: pd.DataFrame, X_array: np.ndarray ,song_id: int) -> List[Tuple] :
    song = df.iloc[song_id]
    X_song = X[song_id]
    _, neighbors = neigh.kneighbors(np.array([X_song]))
    song_list = []
    for idx in neighbors[0][2:]: 
        row = df.iloc[idx]
       # print(f'Artist: {row.artist_name} - Track: {row.track_name}')
        song_list.append((row.artist_name, row.track_name))
    only_10 = random.sample(song_list, 10)
    return(only_10)

In [43]:
# example of function call 
closest_ten(df, X, 9027)

[('Smith Westerns', 'Weekend'),
 ('The Cat Empire', 'Til The Ocean Takes Us All'),
 ('Sick Puppies', "There's No Going Back"),
 ('Red Hot Chili Peppers', 'Higher Ground - Remastered'),
 ('Red Velvet', 'Power Up'),
 ('Ballyhoo!', 'Battle Cry'),
 ('Big Smo', 'Where You From'),
 ('ATEEZ', 'HALA HALA (Hearts Awakened, Live Alive)'),
 ('311', 'Freeze Time'),
 ('Dounia', 'Menace')]

# PCA -> NN

In [32]:
# make copy to transform in case you want to do stuff with the other one later in notebook
df_copy = df[features].copy()

In [33]:
from numpy import array, mean, std, cov
from numpy.linalg import eig
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [34]:
means = df_copy.mean()
# print("\n Means:", means)
scaler = StandardScaler()

In [35]:
pca_X = scaler.fit_transform(df_copy.values)
pca = PCA(0.95)
df_pca = pca.fit(pca_X)

In [36]:
print(pca.explained_variance_)

[4.45794341 2.6619331  1.73667879 1.51142396 1.37838907 1.2611412
 1.13150022 1.10768792 1.07033582 1.06444661 1.06026414 1.05531529
 1.05322221 1.05195636 1.0498224  1.04842724 1.04708279 1.04401482
 1.03562486 1.02767329 1.02411178 1.01842098 1.01476388 1.01108687
 1.00834383 1.00137027 0.98850912 0.92596955 0.82414355 0.79806926
 0.75307744 0.56700277 0.51077434]


In [37]:
%time
pca_neigh = NearestNeighbors(n_neighbors=50)
pca_neigh.fit(pca_X) # NN doesn't need to fit Y

Wall time: 0 ns


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=50, p=2,
                 radius=1.0)

In [ ]:
# refactor later if have time
# from sklearn.pipeline import make_pipeline
# pipeline = make_pipeline(
#     StandardScaler(),
#     NearestNeighbors())

# pipeline.fit(X)

# Auto Encoder -> NN

In [ ]:
from keras.models import Model

In [ ]:
encoding_dim = 10 # compression of factor 4.1 assuming input of 41

# Export Model with Joblib

In [ ]:
filename = 'NearestNeighborGenres.sav'

In [ ]:
joblib.dump(neigh, filename)

In [ ]:
pca_filename = 'PCANearestNeighborGenres.sav'
joblib.dump(pca_neigh, pca_filename)

In [ ]:
# one hot encoding, rolled back
# def encode_and_bind(original_dataframe, feature_to_encode):
#     dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
#     res = pd.concat([original_dataframe, dummies], axis=1)
#     return(res)

# df = encode_and_bind(df, 'key')
# df = encode_and_bind(df, 'mode')
# df = encode_and_bind(df, 'time_signature')

# Export Model with Pickle

In [ ]:
import pickle

model = neigh

filename = 'filename.pkl'

with open(filename, 'wb') as f:
    pickle.dump(model, f)
#then send this pkl file to DE team

#DE team:
#Make sure to save the pkl file in the same folder as the app

#To read it into the app:

# with open(filename, 'rb') as f:
#     model = pickle.load(f)